# Data Aggregation Scripts
This section aggregates the cleaned `trips_data` and `station_data` for further analysis and visualisation

Datasets created:
1. `merged_station_data` looks at the number of trips made to and from a station
2. `journey_data` aggregates station and trips data based on the combination of start and end stations to find out what are the popular bike routes and its aggregated data (using mean and mode etc)
3. `diff_start_end_journey_data` is a subset of `journey_data` where the starting and ending stations are different
4. `same_start_end_journey_data` is a subset of `journey_data` where the starting and ending stations are different
5. `origin_dest_data` takes information from `diff_start_end_journey_data` to visualise which are the common paths taken by users on Tableau (Reference: https://help.tableau.com/current/pro/desktop/en-us/maps_howto_origin_destination.htm#Path)

In [7]:
## Change to True if you want to download aggregated data 
download_files = False 

# Import libraries
import numpy as np
import pandas as pd
import statistics as st

# Import data from data folder
trips_data = pd.read_csv("../data/trips_data.csv")
station_data = pd.read_csv("../data/austin_bikeshare_stations.csv")

### 1. `merged_station_data`

In [8]:
start_station_data = trips_data.groupby(['start_station_name']).agg({'month':len}).sort_values(['month'], ascending=False)
start_station_data = start_station_data.rename({'month':'start_count'}, axis = 1)

end_station_data = trips_data.groupby(['end_station_name']).agg({'month':len}).sort_values(['month'], ascending=False)
end_station_data = end_station_data.rename({'month':'end_count'}, axis = 1)
# to check if it is accurate: sum(start_station_data['start_count']) # end = 615092, start = 615092 => accurate 

merged_station_data = station_data.merge(start_station_data, left_on = "name", right_on="start_station_name", how = 'left')
merged_station_data = merged_station_data.merge(end_station_data, left_on = "name", right_on="end_station_name", how = 'left')
merged_station_data # 72 rows with start_count and end_count denoting how many times users have started or ended their journey at the station

,latitude,location,longitude,name,station_id,status,start_count,end_count
0,30.27041,(30.27041 -97.75046),-97.75046,West & 6th St.,2537,active,11689,13783
1,30.26452,(30.26452 -97.7712),-97.77120,Barton Springs Pool,2572,active,11810,10733
2,30.27595,(30.27595 -97.74739),-97.74739,ACC - Rio Grande & 12th,2545,closed,1749,1309
3,30.28480,(30.2848 -97.72756),-97.72756,Red River & LBJ Library,1004,closed,576,539
4,30.26694,(30.26694 -97.74939),-97.74939,Nueces @ 3rd,1008,moved,4307,4163
...,...,...,...,...,...,...,...,...
67,30.26634,(30.26634 -97.74378),-97.74378,4th & Congress,2495,active,24392,27380
68,30.26780,(30.2678 -97.75189),-97.75189,3rd & West,2552,active,15342,13180
69,30.26960,(30.2696 -97.75332),-97.75332,5th & Bowie,2501,active,26103,24624
70,30.26416,(30.26416 -97.73289),-97.73289,5th & San Marcos,2564,moved,7187,6999


### 2. `journey_data`

In [9]:
journey_data = trips_data.groupby(['start_station_name', 'end_station_name']).agg(\
                          {'bikeid' : len,
                           'day' : lambda x: st.mode(x),
                           'parts_of_day': lambda x: st.mode(x),
                           'duration_minutes': np.mean,
                           'month': lambda x: st.mode(x),
                           'subscriber_type': lambda x: st.mode(x),
                           'year': np.mean,
                           'start_hour': np.mean}).sort_values(by = ['bikeid'], ascending = False)

journey_data['count'] = journey_data['bikeid']
journey_data = journey_data.drop(['bikeid'], axis=1)
journey_data # 4354 rows with aggregated information

day  \
start_station_name                end_station_name                         
Riverside @ S. Lamar              Riverside @ S. Lamar                 6   
Rainey St @ Cummings              Rainey St @ Cummings                 5   
2nd & Congress                    2nd & Congress                       5   
City Hall / Lavaca & 2nd          City Hall / Lavaca & 2nd             5   
Capitol Station / Congress & 11th Capitol Station / Congress & 11th    6   
...                                                                  ...   
ACC - Rio Grande & 12th           6th & Navasota St.                   1   
                                  East 6th & Pedernales St.            5   
East 11th St. at Victory Grill    Pease Park                           0   
Rainey @ River St                 Brazos & 6th                         6   
Republic Square                   East 11th St. & San Marcos           3   

                                                                        parts_of_day  \
start_station_name                end_station_name                                     
Riverside @ S. Lamar              Riverside @ S. Lamar               early afternoon   
Rainey St @ Cummings              Rainey St @ Cummings               early afternoon   
2nd & Congress                    2nd & Congress                     early afternoon   
City Hall / Lavaca & 2nd          City Hall / Lavaca & 2nd           early afternoon   
Capitol Station / Congress & 11th Capitol Station / Congress & 11th   late afternoon   
...                                                                              ...   
ACC - Rio Grande & 12th           6th & Navasota St.                         evening   
                                  East 6th & Pedernales St.           late afternoon   
East 11th St. at Victory Grill    Pease Park                          late afternoon   
Rainey @ River St                 Brazos & 6th                       early afternoon   
Republic Square                   East 11th St. & San Marcos            late morning   

                                                                     duration_minutes  \
start_station_name                end_station_name                                      
Riverside @ S. Lamar              Riverside @ S. Lamar                      49.109894   
Rainey St @ Cummings              Rainey St @ Cummings                      55.968623   
2nd & Congress                    2nd & Congress                            64.688035   
City Hall / Lavaca & 2nd          City Hall / Lavaca & 2nd                  58.751687   
Capitol Station / Congress & 11th Capitol Station / Congress & 11th         44.236900   
...                                                                               ...   
ACC - Rio Grande & 12th           6th & Navasota St.                        61.000000   
                                  East 6th & Pedernales St.                140.000000   
East 11th St. at Victory Grill    Pease Park                                42.000000   
Rainey @ River St                 Brazos & 6th                               0.000000   
Republic Square                   East 11th St. & San Marcos                10.000000   

                                                                     month  \
start_station_name                end_station_name                           
Riverside @ S. Lamar              Riverside @ S. Lamar                   7   
Rainey St @ Cummings              Rainey St @ Cummings                   7   
2nd & Congress                    2nd & Congress                         5   
City Hall / Lavaca & 2nd          City Hall / Lavaca & 2nd               3   
Capitol Station / Congress & 11th Capitol Station / Congress & 11th      7   
...                                                                    ...   
ACC - Rio Grande & 12th           6th & Navasota St.                     4   
                                  East 6th & Pedernales St.              4 

### 3. `diff_start_end_journey_data`

In [10]:
diff_start_end_bool = list(map(lambda tup: True if tup[0] != tup[1] else False,journey_data.index))
diff_start_end_journey_data = journey_data[diff_start_end_bool]
diff_start_end_journey_data # 4282 rows

day  \
start_station_name                      end_station_name                  
5th & Bowie                             4th & Congress                3   
4th & Congress                          5th & Bowie                   1   
5th & Bowie                             City Hall / Lavaca & 2nd      0   
3rd & West                              City Hall / Lavaca & 2nd      4   
Convention Center / 4th St. @ MetroRail City Hall / Lavaca & 2nd      2   
...                                                                 ...   
ACC - Rio Grande & 12th                 6th & Navasota St.            1   
                                        East 6th & Pedernales St.     5   
East 11th St. at Victory Grill          Pease Park                    0   
Rainey @ River St                       Brazos & 6th                  6   
Republic Square                         East 11th St. & San Marcos    3   

                                                                       parts_of_day  \
start_station_name                      end_station_name                              
5th & Bowie                             4th & Congress                early morning   
4th & Congress                          5th & Bowie                  late afternoon   
5th & Bowie                             City Hall / Lavaca & 2nd     late afternoon   
3rd & West                              City Hall / Lavaca & 2nd       late morning   
Convention Center / 4th St. @ MetroRail City Hall / Lavaca & 2nd      early morning   
...                                                                             ...   
ACC - Rio Grande & 12th                 6th & Navasota St.                  evening   
                                        East 6th & Pedernales St.    late afternoon   
East 11th St. at Victory Grill          Pease Park                   late afternoon   
Rainey @ River St                       Brazos & 6th                early afternoon   
Republic Square                         East 11th St. & San Marcos     late morning   

                                                                    duration_minutes  \
start_station_name                      end_station_name                               
5th & Bowie                             4th & Congress                      8.947801   
4th & Congress                          5th & Bowie                        11.289637   
5th & Bowie                             City Hall / Lavaca & 2nd           11.005179   
3rd & West                              City Hall / Lavaca & 2nd            8.807835   
Convention Center / 4th St. @ MetroRail City Hall / Lavaca & 2nd           13.819351   
...                                                                              ...   
ACC - Rio Grande & 12th                 6th & Navasota St.                 61.000000   
                                        East 6th & Pedernales St.         140.000000   
East 11th St. at Victory Grill          Pease Park                         42.000000   
Rainey @ River St                       Brazos & 6th                        0.000000   
Republic Square                         East 11th St. & San Marcos         10.000000   

                                                                    month  \
start_station_name                      end_station_name                    
5th & Bowie                             4th & Congress                  1   
4th & Congress                          5th & Bowie                     3   
5th & Bowie                             City Hall / Lavaca & 2nd        3   
3rd & West                              City Hall / Lavaca & 2nd        3   
Convention Center / 4th St. @ MetroRail City Hall / Lavaca & 2nd        3   
...                                                                   ...   
ACC - Rio Grande & 12th                 6th & Navasota St.              4   
                                        East 6th & Pedernales St.       4   
East 11th St. at Victory Grill          Pease

### 4. `same_start_end_journey_data`

In [11]:
same_start_end_bool = list(map(lambda tup: True if tup[0] == tup[1] else False,journey_data.index))
same_start_end_journey_data = journey_data[same_start_end_bool]
same_start_end_journey_data ## 72 rows -- a much smaller proportion of journey_data

,,day,parts_of_day,duration_minutes,month,subscriber_type,year,start_hour,count
start_station_name,end_station_name,,,,,,,,
Riverside @ S. Lamar,Riverside @ S. Lamar,6,early afternoon,49.109894,7,Walk Up,2015.456959,14.908322,8399
Rainey St @ Cummings,Rainey St @ Cummings,5,early afternoon,55.968623,7,Walk Up,2015.611493,14.173453,5673
2nd & Congress,2nd & Congress,5,early afternoon,64.688035,5,Walk Up,2015.417033,13.461467,4321
City Hall / Lavaca & 2nd,City Hall / Lavaca & 2nd,5,early afternoon,58.751687,3,Walk Up,2015.151501,13.902723,4297
Capitol Station / Congress & 11th,Capitol Station / Congress & 11th,6,late afternoon,44.236900,7,Walk Up,2015.097077,14.570601,3626
...,...,...,...,...,...,...,...,...,...
Congress & Cesar Chavez,Congress & Cesar Chavez,5,early afternoon,41.862745,7,Walk Up,2017.000000,12.588235,51
State Parking Garage @ Brazos & 18th,State Parking Garage @ Brazos & 18th,1,late afternoon,34.470588,3,24-Hour Kiosk (Austin B-cycle),2014.000000,14.039216,51
8th & Guadalupe,8th & Guadalupe,6,late afternoon,36.347826,5,24-Hour Kiosk (Austin B-cycle),2014.000000,13.086957,46


### `origin_dest_data`

In [12]:
## Create two dataframes, one for origin and one for destination from diff_start_end_journey_data
origin_data = diff_start_end_journey_data.copy()
destination_data = diff_start_end_journey_data.copy()

## Manipulate columns to include station names, path_id and origin/destination label
origin_data = origin_data.reset_index()
origin_data = origin_data.drop(['end_station_name'], axis=1)
origin_data['path_id'] = range(len(diff_start_end_journey_data))
origin_data['origin-dest'] = list(['origin']*len(diff_start_end_journey_data))

destination_data = destination_data.reset_index()
destination_data = destination_data.drop(['start_station_name'], axis=1)
destination_data['origin-dest'] = pd.Series(['destination']*len(diff_start_end_journey_data))
destination_data['path_id'] = pd.Series(range(len(diff_start_end_journey_data)))

## Merge each dataset with information from 'merged_station_data'
origin_data = origin_data.merge(merged_station_data, left_on = "start_station_name", right_on = "name", how = "left", suffixes=('_start', '_end'))
destination_data = destination_data.merge(merged_station_data, left_on = "end_station_name", right_on = "name", how = "left", suffixes = ('_start', '_end'))

## Concatenate origin_data and destination_data
origin_dest_data = pd.concat([origin_data, destination_data], axis = 0)
origin_dest_data = origin_dest_data.drop(['start_station_name', 'end_station_name'], axis = 1)
origin_dest_data = origin_dest_data.sort_values(by=['count','path_id','origin-dest'], axis = 0, ascending = False)
origin_dest_data #8564 rows with additional columns such as path_id and origin-dest

,day,parts_of_day,duration_minutes,month,subscriber_type,year,start_hour,count,path_id,origin-dest,latitude,location,longitude,name,station_id,status,start_count,end_count
0,3,early morning,8.947801,1,Local365,2015.309083,11.785450,2433,0,origin,30.26960,(30.2696 -97.75332),-97.75332,5th & Bowie,2501,active,26103,24624
0,3,early morning,8.947801,1,Local365,2015.309083,11.785450,2433,0,destination,30.26634,(30.26634 -97.74378),-97.74378,4th & Congress,2495,active,24392,27380
1,1,late afternoon,11.289637,3,Local365,2015.360939,15.014172,2258,1,origin,30.26634,(30.26634 -97.74378),-97.74378,4th & Congress,2495,active,24392,27380
1,1,late afternoon,11.289637,3,Local365,2015.360939,15.014172,2258,1,destination,30.26960,(30.2696 -97.75332),-97.75332,5th & Bowie,2501,active,26103,24624
2,0,late afternoon,11.005179,3,Local365,2015.175612,14.809322,2124,2,origin,30.26960,(30.2696 -97.75332),-97.75332,5th & Bowie,2501,active,26103,24624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4142,3,night,8.000000,1,Local365,2017.000000,21.000000,1,4142,destination,30.26416,(30.26416 -97.73289),-97.73289,5th & San Marcos,2564,moved,7187,6999
4141,5,late morning,62.000000,7,Walk Up,2017.000000,11.000000,1,4141,origin,30.25987,(30.25987 -97.72373),-97.72373,East 4th & Chicon,3292,active,3343,3536
4141,5,late morning,62.000000,7,Walk Up,2017.000000,11.000000,1,4141,destination,30.27217,(30.27217 -97.75246),-97.75246,Henderson & 9th,3685,active,96,103
4140,4,night,37.000000,9,Local365,2016.000000,21.000000,1,4140,origin,30.26590,(30.2659 -97.76822),-97.76822,Zilker Park,2574,active,13302,15238


## Download data

In [13]:
if download_files:
  merged_station_data.to_csv("../data/merged_station_data.csv")
  journey_data.to_csv("../data/journey_data.csv")
  diff_start_end_journey_data.to_csv("../data/diff_start_end_journey_data.csv")
  same_start_end_journey_data.to_csv("../data/same_start_end_journey_data.csv")
  origin_dest_data.to_csv("../data/origin_dest_data.csv")